Autor: Matyáš Sládek <br>
Rok: 2020 <br>

Tento soubor slouží k extrakci atributů ze skladeb datových sad či lokálního hudebního archivu. <br>
Postupujte podle návodů u jednotlivých buňěk <br>

Tato buňka importuje potřebné knihovny.

In [ ]:
%%capture
import os
import sys
import warnings
import multiprocessing as mp
import time
from datetime import timedelta
import re

import numpy as np
import pandas as pd
from scipy import stats
import soundfile as sf

import librosa
import essentia.standard as es

from tqdm.notebook import tqdm   # Progress bars
tqdm().pandas()

Tato buňka obsahuje funkci pro tisk na obrazovku a zároveň do souboru <strong>feature_extraction.out</strong>

In [ ]:
def print_log(*args, **kwargs):
    """
    Prints to stdout and logs to file.
    """
    print(*args, **kwargs)   # Print to stdout
    with open('../metadata/misc/feature_extraction.out','a') as file:
        print(re.sub('\\033\[.m', '', *args), file=file)   # Print to log file with removed bold text, that would not display correctly

Tato buňka extrahuje atributy pomocí funkcí knihovny <strong>Librosa</strong>. <br>
Zde je možné upravit parametry extrakčních funkcí či nějaké extrakční funkce přidat či odstranit. <br>
Délka okna pro převod signálu do frekvenční oblasti metodami Short-time Fourier transform a Constant-Q transform je nastavena na hodnotu 2048 vzorků a posun oken na hodnotu 1024 vzorků. <br>
Každá skladba je načtena se vzorkovací frekvencí 44100 Hz, skladby s jinou vzorkovací frekvencí jsou případně převzorkovány, aby nebylo nutné upavovat parametry extrakčních funkcí. <br>
Detailní popis parametrů a funkcí této knihovny je možné nalézt zde <https://librosa.org/doc/latest/index.html>  a některých, včetně vizualizace zde <https://musicinformationretrieval.com/>. <br>
Celkem je extrahováno 14 atributů, a to: <br>
<ul>
    <li><strong>zero_crossing_rate</strong> - Počet průchodů signálu nulovou úrovní (změna znaménka u amplitudy)</li>
    <li><strong>chroma_cens</strong> - Normalizované statistiky energií tónů chromatické stupnice</li>
    <li><strong>chroma_cqt</strong> - Constant-Q spektrogram s pásmy chromatické stupnice</li>
    <li><strong>tonnetz</strong> - Intonace not</li>
    <li><strong>rms</strong> - Energie signálu</li>
    <li><strong>spectral_centroid</strong> - Geometrický střed frekvenčního spektra</li>
    <li><strong>spectral_bandwidth</strong> - Šířka distribuce frekvencí</li>
    <li><strong>spectral_contrast</strong> - Rozdíl energie vrcholů a údolí spektra</li>
    <li><strong>spectral_flatness</strong> - Drsnost frekvenčního spektra (je signál více jako tón či hluk)</li>
    <li><strong>spectral_rolloff</strong> - Frekvence, pod kterou spadá 85 % energie spektra</li>
    <li><strong>stft</strong> - Spektrogram</li>
    <li><strong>mel_spectrogram</strong> - Spektrogram s frekvenční osou převedenou do Mel-frekvenční škály</li>
    <li><strong>mfcc</strong> - Mel-frekvenční kepstrální koeficienty</li>
    <li><strong>tempo</strong> - Tempo</li>
</ul>

Všechy extrahované atributy jsou ve formátu: <br>
[název_atributu] [statistická_funkce] [číslo_hodnoty] [hodnota_atributu] <br>
Skladba je identifikována cestou k souboru jeho názvem uloženém v názvu objektu Pandas Series. <br>

In [ ]:
def extract_features_librosa(filepath):
    """
    Extracts features using Librosa library.

    Parameters:

    filepath: path to the audio file
    """
    def process_feature(feature_name):
        """
        Transforms extracted feature into indexed format used to create pandas series. Statistic functions are used to reduce dimensionality when needed.
        
        Parameters:
        
        feature_name: name of the feature
        """
        if feature.size > 1:   # If a feature is a list or list of lists, process it with statistic functions to reduce dimensionality
            
            # For each statistic function, process the feature with it and create frst two rows of multiindex
            for statistic_name, statistic in statistics.items():
                processed_feature = statistic(feature, axis=1)
                index = [feature_name, statistic_name]

                if isinstance(processed_feature, (np.ndarray)):   # If processed feature has multiple values
                    
                    # For each value of the feature, append unique number to multiindex
                    for i, processed_feature_i in enumerate(processed_feature):
                        index.append('{:03d}'.format(i+1))

                        # Append multiindex to indices list
                        for j in range(3):
                            indices[j].append(index[j])

                        data.append('{:.7e}'.format(processed_feature_i))   # Append processed feature values to data list
                        index = index[:-1]   # Reset last row of multiindex
                else:   #If processed feature has one value
                    index.append('001')   # Append unique value to multiindex

                    # Append multiindex to indices list
                    for j in range(3):
                        indices[j].append(index[j]) 

                    data.append('{:.7e}'.format(processed_feature))   # Append processed feature values to data list                    
        else:   # If feature is a single value, do not process it with statistic functions
            index = [feature_name, 'none', '001']   # Create multiindex
            
            # Append multiindex to indices list
            for j in range(3):
                indices[j].append(index[j])

            data.append('{:.7e}'.format(feature[0]))   # Append processed feature values to data list

        return 
        
    # Statistic functions to be used, unwanted can be commented out   
    statistics = {
        'mean':np.mean,
        'std':np.std,
        'median':np.median,
        'min':np.min,
        'max':np.max,
        'skew':stats.skew,
        'kurtosis':stats.kurtosis,
    }      
    data = []   # Stores feature values
    indices = [[], [], []]   # Stores indices of feature values for pandas series
    names = ['feature', 'statistic', 'number']   # Names of indices

    # Load track 
    try:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            y, sr = librosa.load(filepath, sr=44100, mono=True)
    except Exception as e:
        print_log('Loading file: "{}" with Librosa library has failed! Error: {}'.format(filepath, repr(e)), file=sys.stderr)
        return
                
    duration = librosa.get_duration(y=y, sr=sr)   # Get track duration

    ###################################################################################################################################################
    
    # Quality control of datasets, comment out if using full length tracks
    
    # Remove tracks with wrong duration (should be 30 s)
    if not ((duration > 29) and (duration < 31)):
        print_log("Track " + filepath + " does not have an approximate duration of 30 s. Its duration is " + str(duration) + " seconds.", file=sys.stderr)
        return
    
    ###################################################################################################################################################

    frame_length = 2048   # Length of a window for frequency analysis corresponds to 46 ms of audio which should be sufficient
    # higher value may be better for more accurate frequency analysis of tonal features, but that is already used in Essentia library, so this value is left for comparison
    hop_length = 1024   # Length of step between windows, set to half of frame length so each frame is fully overlaped by adjacent windows

    # Extract zero crossing rate feature (rate of change in amplitude sign)
    feature = librosa.feature.zero_crossing_rate(y, frame_length=frame_length, hop_length=hop_length)
    process_feature('zero_crossing_rate')

    # Compute constant-Q transform (similiar to STFT, adjusts band size to logarithmic to reflect human hearing) 7 octaves of 12 bins used (default)
    # Hanning window function to smooth out signal at frame borders
    C = np.abs(librosa.cqt(y, sr=sr, hop_length=hop_length, n_bins=84, bins_per_octave=12, tuning=None, window='hann', scale=True))

    # Extract chroma cens feature, 7 octaves with 12 notes each to reflect chromatic scale (used for note/melody detection)
    feature = librosa.feature.chroma_cens(C=C, n_chroma=12, n_octaves=7)
    process_feature('chroma_cens')

    # Extract chroma cqt feature, 7 octaves with 12 notes each to reflect chromatic scale (used for note/melody detection)
    feature = librosa.feature.chroma_cqt(C=C, n_chroma=12, n_octaves=7)
    process_feature('chroma_cqt')

    # Extract tonnetz feature, used to detect intonation
    feature = librosa.feature.tonnetz(chroma=feature)
    process_feature('tonnetz')   

    # Compute short-time Fourier transform
    # Hanning window function to smooth out signal at frame borders
    S = np.abs(librosa.stft(y, n_fft=frame_length, hop_length=hop_length, window='hann'))

    # Extract root mean square feature (signal energy)
    feature = librosa.feature.rms(S=S)
    process_feature('rms')

    # Extract spectral centroid feature (geometrical center of frequency distribution)
    feature = librosa.feature.spectral_centroid(S=S)
    process_feature('spectral_centroid')

    # Extract spectral bandwidth feature (measures spread of distribution of frequencies)
    feature = librosa.feature.spectral_bandwidth(S=S)
    process_feature('spectral_bandwidth')

    # Extract spectral contrast feature (energy difference between crests and troughs)
    feature = librosa.feature.spectral_contrast(S=S, sr=sr)
    process_feature('spectral_contrast')

    # Extract spectral flatness feature (measures noisiness of signal)
    feature = librosa.feature.spectral_flatness(S=S)
    process_feature('spectral_flatness')

    # Extract spectral rolloff feature (measures frequency under which 85 % of the spectral energy lies)
    feature = librosa.feature.spectral_rolloff(S=S, sr=sr, roll_percent=0.85)
    process_feature('spectral_rolloff')

    # Transform magnitude spectrogram to power spectrogram
    S = S**2

    # Extract chroma short time Fourier transform (similiar to chroma_CQT, but has constant frequency band size)
    feature = librosa.feature.chroma_stft(S=S, sr=sr, tuning=None, n_chroma=12)
    process_feature('chroma_stft')

    # Extract mel-spectrogram (spectrogram in mel-frequencz scale to reflect human hearing, linear to 1 kHz than logarithmic)
    feature = librosa.feature.melspectrogram(S=S, sr=sr, n_mels=128)
    process_feature('melspectrogram')

    # Extract mel-frequency cepstral coefficients (compact form of mel-spectrogram)
    feature = librosa.feature.mfcc(S=librosa.power_to_db(feature), n_mfcc=20, dct_type=2, norm='ortho')
    process_feature('mfcc')

    # Compute onset strength envelope (used to detect sudden changes in amplitude)
    onset_env = librosa.onset.onset_strength(y, sr=sr)

    # Extract estimated tempo
    feature = librosa.beat.tempo(onset_envelope=onset_env, sr=sr)
    process_feature('tempo')

    features = pd.Series(data=data, index=pd.MultiIndex.from_arrays(indices, names=names))   # Transform processed features to pandas series
    features.name = filepath   # Create index for the series of features

    return features

Tato buňka extrahuje atributy pomocí funkcí knihovny <strong>Essentia</strong>. <br>
Zde je možné upravit parametry extrakční funkce. <br>

Hodnoty parametrů extrakční funkce: <br>
<ul>
    <li><strong>'analysisSampleRate': 44100</strong> - Vzorkovací frekvence skladeb pro analýzu, 44100 Hz (CD kvalita), skladby jsou případně převzorkovány.</li>
    <li><strong>'endTime': 1e+06</strong> - Jak dlouhý úsek skladby má být analyzován, nastaveno na celou skladbu.</li>
    <li><strong>'gfccStats': ["mean", "cov", "icov"]</strong> - Jaké statistické funkce použít pro zpracování hodnot gamma tónových kepstrálních koeficientů, nastaveno na průměr, kovarianci a inverzní kovarianci.</li>
    <li><strong>'loudnessFrameSize': 88200</strong> - Délka okna pro časově-frekvenční analýzu pro odhad hlasitosti, nastaveno na hodnotu 88200 vzorků, což odpovídá úseku dlouhému 2 sekundy. Delší okno je vhodné pro přesnější odhad.</li>
    <li><strong>'loudnessHopSize': 44100</strong> - Posun  okna pro časově-frekvenční analýzu pro odhad hlasitosti, nastaveno na délku poloviny okna, takže se každé okno z poloviny překrývá z oknem předchozím.</li>
    <li><strong>'lowlevelFrameSize': 2048</strong> - Délka okna pro časově-frekvenční analýzu pro výpočet nízko-úrovňových atributů, nastaveno na hodnotu 2048 vzorků, což odpovídá úseku dlouhému 46 milisekund. Kratší okno je vhodné pro přesnější časovou analýzu.</li>
    <li><strong>'lowlevelHopSize': 1024</strong> - Posun okna pro časově-frekvenční analýzu pro výpočet nízko-úrovňových atributů, nastaveno na délku poloviny okna, takže se každé okno z poloviny překrývá z oknem předchozím.</li>
    <li><strong>'lowlevelSilentFrames': 'noise'</strong> - Zpracování oken neobsahujících zvuk, ponechána výchozí hodnota 'noise' - doplnění šumem.</li>
    <li><strong>'lowlevelStats': ["mean", "var", "stdev", "median", "min", "max", "dmean", "dmean2", "dvar", "dvar2"]</strong> - Jaké statistické funkce použít pro zpracování hodnot nízkoúrovňových atributů, nastaveno na průměr, rozptyl, směrodatnou odchylku, medián, minimální hodnotu, maximální hodnotu, průměr první derivace, průměr druhé derivace, rozptyl první derivace a rozptyl druhé derivace.</li>
    <li><strong>'lowlevelWindowType': 'blackmanharris62'</strong> - Typ okénkové funkce pro zpracování oken spektrogramu pro nízkoúrovňové atributy. Nastaveno typ blackmanharris62.</li>
    <li><strong>'lowlevelZeroPadding': 0</strong> - Odsazení oken pro nízkoúrovňové atributy, nepoužito.</li>
    <li><strong>'mfccStats': ["mean", "cov", "icov"]</strong> - Jaké statistické funkce použít pro zpracování hodnot Mel-frekvenčních kepstrálních koeficientů, nastaveno na průměr, kovarianci a inverzní kovarianci.</li>
    <li><strong>'profile': ''</strong> - Profil pro načtení parametrů, nepoužito.</li>
    <li><strong>'requireMbid': False</strong> - Vyžadovat tag MusicBrainz databáze, nepoužito.</li>
    <li><strong>'rhythmMaxTempo': 208</strong> - Maximální hodnota tempa, nastaveno na hodnotu 208 úderů za minutu, což by mělo zamezit odhadům tempa ze čtvrtdob a podobně.</li>
    <li><strong>'rhythmMethod': 'degara'</strong> - Metoda pro odhad tempa, nastaveno na metodu degara.</li>
    <li><strong>'rhythmMinTempo': 40</strong> - Minimální hodnota tempa, nastaveno na hodnotu 40 úderů za minutu.</li>
    <li><strong>'rhythmStats': ["mean", "var", "stdev", "median", "min", "max", "dmean", "dmean2", "dvar", "dvar2"]</strong> - Jaké statistické funkce použít pro zpracování hodnot rytmických atributů, nastaveno na průměr, rozptyl, směrodatnou odchylku, medián, minimální hodnotu, maximální hodnotu, průměr první derivace, průměr druhé derivace, rozptyl první derivace a rozptyl druhé derivace.</li>
    <li><strong>'startTime': 0</strong> - Od jakého času začít analýzu, nastaveno na hodnotu 0 sekund.</li>
    <li><strong>'tonalFrameSize': 4096</strong> - Délka okna pro časově-frekvenční analýzu pro výpočet melodických atributů, nastaveno na hodnotu 4096 vzorků, což odpovídá úseku dlouhému 92 milisekund. Delší okno je vhodné pro přesnější frekvenční odhad.</li>
    <li><strong>'tonalHopSize': 2048</strong> - Posun okna pro časově-frekvenční analýzu pro výpočet melodických atributů, nastaveno na délku poloviny okna, takže se každé okno z poloviny překrývá z oknem předchozím.</li>
    <li><strong>'tonalSilentFrames': 'noise'</strong> - Zpracování oken neobsahujících zvuk, ponechána výchozí hodnota 'noise' - doplnění šumem.</li>
    <li><strong>'tonalStats': ["mean", "var", "stdev", "median", "min", "max", "dmean", "dmean2", "dvar", "dvar2"]</strong> - Jaké statistické funkce použít pro zpracování hodnot melodických atributů, nastaveno na průměr, rozptyl, směrodatnou odchylku, medián, minimální hodnotu, maximální hodnotu, průměr první derivace, průměr druhé derivace, rozptyl první derivace a rozptyl druhé derivace.</li>
    <li><strong>'tonalWindowType': 'blackmanharris62'</strong> - Typ okénkové funkce pro zpracování oken spektrogramu pro melodické atributy. Nastaveno typ blackmanharris62.</li>
    <li><strong>'tonalZeroPadding': 0</strong> - Odsazení oken pro melodické atributy, nepoužito.</li>
</ul>

Každá skladba je načtena se vzorkovací frekvencí 44100 Hz, skladby s jinou vzorkovací frekvencí jsou případně převzorkovány, aby nebylo nutné upavovat parametry extrakčních funkcí. <br>
Detailní popis extrakční funkce, parametrů a atributů této knihovny je možné nalézt zde <https://essentia.upf.edu/reference/std_MusicExtractor.html> <br>
Celkem je extrahováno 80 atributů, z toho 49 nízkoúrovňových, 14 rytmických a 17 melodických/harmonických. <br>
Všechy extrahované atributy jsou ve formátu: <br>
[úroveň_atributu][název_atributu] [statistická_funkce] [číslo_hodnoty] [hodnota_atributu] <br>
Skladba je identifikována cestou k souboru jeho názvem uloženém v názvu objektu Pandas Series. <br>

In [ ]:
def extract_features_essentia(filepath):
    """
    Extracts features using Essentia library.

    Parameters:

    filepath: path to the audio file
    """
    # Parameters used by Essentia music extractor (see Essentia docs)
    params = {
        'analysisSampleRate': 44100,   # Sample rate for the analysis, tracks are resampled if necessary
        'endTime': 1e+06,   # How much of the track should be loaded, set to high number to fully load tracks (each has 30s)
        'gfccStats': ["mean", "cov", "icov"],   # Which statistc functions to use for dimensionality reduction of gammatone frequency cepstral coefficients, mean, covariance and inverse covariance are used 
        'loudnessFrameSize': 88200,   # Frame size of STFT window for estimating loudness, higher number is needed for more accurate estimation (corresponds to 2 sec of audio)
        'loudnessHopSize': 44100,   # Hop size is set to half the frame size so that each frame is fully covered by adjacent frames to avoid incorrect estimations on window borders
        'lowlevelFrameSize': 2048,   # Frame size of STFT window for extraction of low level features, set to lower value to have more time accurate analysis (corresponds to 46 miliseconds of audio)
        'lowlevelHopSize': 1024,   # Hop size is set to half the frame size so that each frame is fully covered by adjacent frames to avoid incorrect estimations on window borders
        'lowlevelSilentFrames': 'noise',   # Replaces frames that only contain silence to random noise
        'lowlevelStats': ["mean", "var", "stdev", "median", "min", "max", "dmean", "dmean2", "dvar", "dvar2"],   # Statistic functions for processing low level features
                        # mean, variance, standard deviation, median, minimum, maximum, mean and variance of first and second derivatives
        'lowlevelWindowType': 'blackmanharris62',   # Windowing function to smooth out frame borders for frequency analysis
        'lowlevelZeroPadding': 0,   # Not used (padding windows with zeroes)
        'mfccStats': ["mean", "cov", "icov"],   # Which statistc functions to use for dimensionality reduction of mel-frequency cepstral coefficients, mean, covariance and inverse covariance are used 
        'profile': '',   # Not used (for overwritting parameter values)
        'requireMbid': False,   # Not used (ignores tracks without MusicBrainz tags)
        'rhythmMaxTempo': 208,   # Mamium value for tempo estimation to reduce detecting half/quarter beats (multiples of actual tempo)
        'rhythmMethod': 'degara',   # Method for tempo estimation
        'rhythmMinTempo': 40,   # Minimum value for tempo detection to avoid detecting fractions of actual tempo
        'rhythmStats': ["mean", "var", "stdev", "median", "min", "max", "dmean", "dmean2", "dvar", "dvar2"],   # Same as low level
        'startTime': 0,   # Start time of analysis se to start of track
        'tonalFrameSize': 4096,   # Frame size of STFT window for extraction of tonal features, set to higher value to have more frequency accurate analysis for note detection (corresponds to 92 miliseconds of audio)
        'tonalHopSize': 2048,   # Hop size is set to half the frame size so that each frame is fully covered by adjacent frames to avoid incorrect estimations on window borders
        'tonalSilentFrames': 'noise',   # Same as low level
        'tonalStats': ["mean", "var", "stdev", "median", "min", "max", "dmean", "dmean2", "dvar", "dvar2"],   # Same as low level
        'tonalWindowType': 'blackmanharris62',   # Same as low level
        'tonalZeroPadding': 0,   # Same as low level
    }
    
    try:
        # If a file is in .au format, create temporary .wav file from it file to be processed with Essentia music extractor (.au is not supported)
        # Then extract features
        if filepath.lower().endswith('.au'):
            data, samplerate = sf.read(filepath)
            tmp_filepath = os.path.join('../data/tmp', '{}.wav'.format(os.path.splitext(os.path.basename(filepath))[0]))
            sf.write(tmp_filepath, data, samplerate)
            results, _ = es.MusicExtractor(**params)(tmp_filepath)
            os.remove(tmp_filepath)
        else:
            results, _ = es.MusicExtractor(**params)(filepath)
    except Exception as e:
        print_log('Extracting features from file: "{}" with Essentia library has failed! Error: {}'.format(filepath, repr(e)), file=sys.stderr)
        return

    data = []   # Stores feature values
    indices = [[], [], [], []]   # Stores indices of feature values for pandas series
    names = ['category', 'feature', 'statistic', 'number']   # Names of indices

    duration = results['metadata.audio_properties.length'] # Get track duration

    ###################################################################################################################################################
    
    # Quality control of datasets, comment this out if using full length tracks
    
    # Remove tracks with wrong duration (should be 30 s)
    if not ((duration > 29) and (duration < 31)):
        print_log("Track " + filepath + " does not have an approximate duration of 30 s. Its duration is " + str(duration) + " seconds.", file=sys.stderr)
        return
    
    ###################################################################################################################################################

    # Transform extracted features to a format suitable for pandas series (statistic functions were already applied by the extractor)
    # For each feature
    for feature_name in results.descriptorNames():
        it = feature_name.split('.')   # Split features description for creating multiindex. Format is: (level, feature_name_first_word, feature_name_second_word, statistics)
        feature = results[feature_name]   # Get the values of the feature

        # If entry is not metadata
        if not 'metadata' in it:
            
            # If feature was not processed with statistics functions, apend 'none' for statistics multiindex row
            if len(it) == 2:
                it.append('none')

            # If feature has two word name, concatenate them together to keep constant level depth of multiindex
            if len(it) == 4:
                it = [it[0], '{}_{}'.format(it[1], it[2]), it[3]]

            if isinstance(feature, (np.ndarray)):   # If feature is an array (can be multidimensional)
                for i, feature_i in enumerate(feature):   # For each value of array
                    if isinstance(feature_i, (np.ndarray)):   # If value is an array
                        
                        # For each value of the feature, append unique number to multiindex
                        for j, feature_i_j in enumerate(feature_i):
                            it.append('{:03d}'.format(j+1+i*len(feature_i)))

                            # Append multiindex to indices list
                            for k in range(4):
                                indices[k].append(it[k])

                            data.append('{:.7e}'.format(feature_i_j) if isinstance(feature_i_j, (np.float32)) else feature_i_j)   # Append feature values to data list
                            it = it[:-1]   # Reset last row of multiindex
                    else:   # If value is a single value
                        it.append('{:03d}'.format(i+1))   # Append unique number to multiindex

                        # Append multiindex to indices list
                        for j in range(4):
                            indices[j].append(it[j])

                        data.append('{:.7e}'.format(feature_i) if isinstance(feature_i, (np.float32)) else feature_i)   # Append feature values to data list
                        it = it[:-1]   # Reset last row of multiindex
            else:   # If feature is a single value
                it.append('001')   # Append unique number to multiindex

                # Append multiindex to indices list
                for i in range(4):
                    indices[i].append(it[i])

                data.append('{:.7e}'.format(feature) if isinstance(feature, (float)) else feature)   # Append feature values to data list

    features = pd.Series(data=data, index=pd.MultiIndex.from_arrays(indices, names=names))   # Transform processed feature to pandas series
    features.name = filepath   # Create index for the series of features

    return features

Tato buňka slouží k vytvoření seznamu cest k souborům z datové sady či archivu a následnému spuštění extrakce pomocí vybrané knihovny. <br>
Cestu ke složkám se skladbami u vlastní datové sady je možné vložit do proměnné <code>datasets</code> ve formátu <code>'název_datové_sady':'cesta_k_datům'</code> <br>
Které extrakční knihovny mají být použity je možné zvolit v proměnné <code>feature_extraction_libraries</code> zakomentováním/odkomentováním příslužného záznamu. <br>
Pro zachování konstantního počtu atributů je možné nastavit proměnnou <code>drop_beats_position_feature</code> na hodnotu True. To je vhodné pro případ, kdy by natrénované klasifikátory byly využity pro predikci dat z jiné datové sady. 

In [ ]:
if __name__ == "__main__":
    
    # Dictionary of names and paths to datasets to be processed, unwanted can be commented out   
    datasets = {
#         'EBD':'../data/EBD',
#         'FMA':'../data/FMA/fma_small',
#         'GTZAN':'../data/GTZAN',
        'local_archive':'../data/local_archive'
    }
    
    #Dictionary of library names and functions to be used to proccess datasets, unwanted can be commented out   
    feature_extraction_libraries = {
        'librosa':extract_features_librosa,
        'essentia':extract_features_essentia,
    }
    
    # Whether to drop beats_position feature of Essentia library
    drop_beats_position_feature = False   # This feature has variable length, which may cause problems when using trained classifiers to predict data from different dataset

    # Create folder for storing features
    if not os.path.exists('../metadata/features'):
        try:
            os.mkdir('../metadata/features')
        except Exception as e:
            print_log('{}: {}'.format('os.mkdir(../metadata/features)', repr(e)), file=sys.stderr)

    t_start = time.time()   # Store start time of extraction process
    
    for dataset_name, path_to_dataset in tqdm(datasets.items(), desc='Datasets extraction progress'):   # For each of the selected datasets
        paths_to_tracks = {}   # Stores tracks indices and paths

        # Walk through every file of a dataset, if it is a track, append its index and path to a dictionary
        for dir_path, _, files in os.walk(path_to_dataset):
            for file in files:
                if file.lower().endswith(('.mp3', '.au')):
                    paths_to_tracks.update({os.path.join(dir_path, file): file})

        # Get number of CPU cores for multiprocessing
        workers = 1 if os.cpu_count() is None else os.cpu_count()

        for library_name, extractor in feature_extraction_libraries.items():   # For each selected extraction library
            
            # Create temporary folder for storing temporary converted track files (essentia does not support loading GTZANs ULAW format)
            if not os.path.exists('../data/tmp'):
                try:
                    os.mkdir('../data/tmp')
                except Exception as e:
                    print_log('{}: {}'.format('os.mkdir(../data/tmp)', repr(e)), file=sys.stderr)

            t = time.time()   # Store start time of extraction process for each library and dataset
            pool = mp.Pool(processes=workers)   # Initialize multiprocessing pool
            print('{} dataset extraction using {} library progress:\n'.format(dataset_name, library_name))
            
            # Commence extraction process and store successful results
            result = [x for x in list(tqdm(pool.imap_unordered(extractor, list(paths_to_tracks.keys())), total=len(paths_to_tracks))) if x is not None]
            print_log('{} dataset extraction using {} library finished in {}.'.format(dataset_name, library_name, str(timedelta(seconds=(time.time() - t))).split(".")[0]))
            pool.close()   # Close multiprocessing pool
            pool.join()   # Join processes

            # If ULAW format had to loaded with Essentia library, remove temporary folder for storing temporary converted track files
            if os.path.exists('../data/tmp'):
                try:
                    os.rmdir('../data/tmp')
                except Exception as e:
                    print_log('{}: {}'.format('os.rmdir(../data/tmp)', repr(e)), file=sys.stderr)
                    
            # Get index of features
            index = result[0].index
            
            # Get shortest index to avoid having sparse dataframe due to variable length of feature beats_position
            for r in result:
                if len(r.index) < len(index):
                    index = r.index
                    
            # Use shortest feature to create dataframe columns, cut longer features to avoid NaN values
            features = pd.DataFrame(index=paths_to_tracks.keys(), columns=index)
            
            # Drop beats_position feature with variable length to avoid having problems when predicting data from a different dataset with trained classifiers
            if drop_beats_position_feature:
                features = features.drop('beats_position', axis=1, level=1)

            # Populate dataframe with feature values
            for track in result:
                features.loc[track.name] = track

#             # Reindex tracks with filenames (Do not use this if you later want to annotate track files with predicted genres)
#             features = features.rename(index=paths_to_tracks)

            # Remove dataframe rows containing NaN or Inf values
            tmp_len = len(features)
            features = features.replace(['-inf', 'inf', 'nan'], np.nan)
            features = features.replace([np.inf, -np.inf], np.nan)
            features = features.dropna()

            print_log("Removed " + str(tmp_len - len(features)) + " corrupted tracks.")
            print_log("Final track count is: " + str(len(features)) + " tracks.\n\n\n")

            # Drop unnecessary first level of multi-index created by Essentia extractor 
            if library_name == 'essentia':
                features.columns = features.columns.droplevel()
                
            # Sort indices of a dataframe for more efficient slicing and save dataframe to .csv file
            features = features.sort_index(axis=0)
            features = features.sort_index(axis=1)
            features.to_csv('../metadata/features/features_{}_{}.csv'.format(dataset_name, library_name))
            
    print_log("Feature extraction for all selected datasets and extraction libraries finished in ~\033[1m{}\033[0m.".format(str(timedelta(seconds=(time.time() - t_start))).split(".")[0]))